# Credit Risk Modelling: Project
Antoni Ballaun, Laura Hoang, Jędrzej Joniec, Piotr Zambrzycki

## 01. Data preparation: missing data

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# import os
# os.getcwd()

In [ ]:
df = pd.read_csv('prosperLoanData.csv')

In [ ]:
df.head()

### Modifying `LoanStatus`

In [ ]:
df['LoanStatus'].unique()

In [ ]:
df['LoanStatus'].value_counts()

If past due > 90 days we it will be considered a default(1), else non-default(0), Canceled will be dropped hard to say what it means and there is only 5 of them 

In [ ]:
values_to_make_ones = ['Past Due (91-120 days)', 'Past Due (>120 days)', 'Defaulted', 'Chargedoff']

In [ ]:
df = df[df['LoanStatus'] != 'Cancelled']

### Creation of `Defaulted` column - our target variable

In [ ]:
df['Defaulted'] = np.where(df['LoanStatus'].isin(values_to_make_ones),1,0)

In [ ]:
sns.countplot(data=df,x='Defaulted', stat='percent',)
# sns.countplot(data=df,x='Defaulted')

### Missing data

#### Removing columns with missing data

In [ ]:
missing_percents = (df.isna().sum().sort_values(ascending=False))/len(df) 

If missing values are greater than 20%, the column will be dropped, imputing without introducing a high level of bias would be very hard.

In [ ]:
print(missing_percents[missing_percents >= 0.2])
list_of_cols_missing_more_than25 = list(missing_percents[missing_percents >= 0.2].index)

In [ ]:
df = df.drop(list_of_cols_missing_more_than25,axis=1)

#### Rows with missing data: removing chosen ones

In [ ]:
missing_percents = (df.isna().sum().sort_values(ascending=False))/len(df)
print(missing_percents[missing_percents > 0])

In [ ]:
missing_amounts = df.isna().sum().sort_values(ascending=False)
missing_amounts[missing_amounts > 0]

In [ ]:
len(df)

There are 113k observations so we will drop missing rows for if less than 1000 is missing

# (DO poprawy !!!)

In [ ]:
missing_amounts_less_than_1000 = missing_amounts[(missing_amounts > 0) & (missing_amounts < 1000)]

In [ ]:
list_of_missing_to_drop_rows = list(missing_amounts_less_than_1000.index)

In [ ]:
list_of_missing_to_drop_rows

In [ ]:
df2 = df[list_of_missing_to_drop_rows].dropna(axis=0,)

In [ ]:
df2.shape

In [ ]:
df2.columns

In [ ]:
df3 = pd.merge(df,df2,how='inner')

In [ ]:
df3.shape

#### Filling rest of missing values: data imputation

In [ ]:
missings_to_process = df3.isnull().sum().sort_values(ascending=False)
missings_to_process = missings_to_process[missings_to_process > 0]

In [ ]:
missings_to_process = list(missings_to_process.index)

In [ ]:
missings_to_process

##### 1) Debt to income ratio
`DebtToIncomeRatio` - the debt to income ratio of the borrower at the time the credit profile was pulled. This value is Null if the debt to income ratio is not available. This value is capped at 10.01 (any debt to income ratio larger than 1000% will be returned as 1001%).

In [ ]:
sns.histplot(data=df3,x='DebtToIncomeRatio',bins=20)

In [ ]:
mean_debt = df3['DebtToIncomeRatio'].mean()

In [ ]:
df3['DebtToIncomeRatio'] = df3['DebtToIncomeRatio'].fillna(value=mean_debt)

EmploymentStatusDuration

In [ ]:
sns.histplot(data=df3,x='EmploymentStatusDuration',bins=10)

In [ ]:
mode_employment = df3['EmploymentStatusDuration'].mode()

In [ ]:
type(mode_employment)

We don't have information, why there is missing data here so we chose the mode.

In [ ]:
df3['EmploymentStatusDuration'] = df3['EmploymentStatusDuration'].fillna(value=0)

In [ ]:
df3['BankcardUtilization'].unique()

In [ ]:
sns.histplot(data=df3,x='BankcardUtilization',bins=10)

In [ ]:
mean_utilization = df3['BankcardUtilization'].mean()

In [ ]:
df3['BankcardUtilization'] = df3['BankcardUtilization'].fillna(value=mean_utilization)

In [ ]:
sns.histplot(data=df3,x='RevolvingCreditBalance',bins=10)

In [ ]:
median_balance = df3['RevolvingCreditBalance'].median()

In [ ]:
df3['RevolvingCreditBalance'].describe()

In [ ]:
df3['RevolvingCreditBalance'] = df3['RevolvingCreditBalance'].fillna(value=median_balance)

In [ ]:
df3['PublicRecordsLast12Months']

In [ ]:
sns.histplot(data=df3,x='PublicRecordsLast12Months',bins=50)

In [ ]:
df3['PublicRecordsLast12Months'].value_counts()

In [ ]:
df3['PublicRecordsLast12Months'] = df3['PublicRecordsLast12Months'].fillna(value=0.0)

OpenCreditLines is the same as Currentcredit line so we will drop one of them

In [ ]:
df3 = df3.drop('CurrentCreditLines',axis=1)

Można wypełnić na podstawie drugiej!

In [ ]:
sns.histplot(data=df3,x='OpenCreditLines',bins=50)

In [ ]:
df3['OpenCreditLines'].value_counts()

In [ ]:
df3['OpenCreditLines'] = df3['OpenCreditLines'].fillna(df3['OpenCreditLines'].mean())

Amount Delinqeunt

In [ ]:
df3['AmountDelinquent'].median()

In [ ]:
df3['AmountDelinquent'] = df3['AmountDelinquent'].fillna(value=0.0)

In [ ]:
df3['TotalTrades']

In [ ]:
sns.histplot(data=df3,x='TotalTrades',bins=50,kde=True)

In [ ]:
df3['TotalTrades'] = df3['TotalTrades'].fillna(value=df3['TotalTrades'].mean())

In [ ]:
df3['TradesOpenedLast6Months'].nunique()

In [ ]:
sns.countplot(data=df3,x='TradesOpenedLast6Months',)

In [ ]:
df3['TradesOpenedLast6Months'] = df3['TradesOpenedLast6Months'].fillna(method='ffill')

In [ ]:
df3['TradesNeverDelinquent (percentage)'].value_counts()

In [ ]:
sns.histplot(data=df3,x='TradesNeverDelinquent (percentage)',bins=10)

In [ ]:
df3['TradesNeverDelinquent (percentage)'].mean()

In [ ]:
df3['TradesNeverDelinquent (percentage)'] = df3['TradesNeverDelinquent (percentage)'].fillna(df3['TradesNeverDelinquent (percentage)'].mean())

In [ ]:
sns.histplot(data=df3,x='AvailableBankcardCredit',bins=10)

In [ ]:
df3['AvailableBankcardCredit'].mean()

In [ ]:
df3['AvailableBankcardCredit'] = df3['AvailableBankcardCredit'].fillna(df3['AvailableBankcardCredit'].mean())

In [ ]:
df3 = df3.drop('Occupation',axis=1)

In [ ]:
df3['BorrowerState'] = df3['BorrowerState'].fillna(method='ffill')

In [ ]:
df3['EmploymentStatus'].unique()

In [ ]:
df3['EmploymentStatus'] = df3['EmploymentStatus'].fillna(method='ffill')

In [ ]:
df3['TotalInquiries'].value_counts()

In [ ]:
sns.histplot(data=df3,x='TotalInquiries',bins=20)

In [ ]:
df3['TotalInquiries'].median()

In [ ]:
df3['TotalInquiries'] = df3['TotalInquiries'].fillna(value=4)

In [ ]:
df3.isnull().sum().sort_values(ascending=False)

## Converting date columns to days since 1970-01-01 and boolean columns to binary values.

In [ ]:
date_columns = [
    'ListingCreationDate',
    'ClosedDate',
    'DateCreditPulled',
    'FirstRecordedCreditLine',
    'LoanOriginationDate'
]

epoch = pd.to_datetime('1970-01-01')
for col in date_columns:
    if col in df3.columns:
        df3[col] = pd.to_datetime(df3[col], errors='coerce')
        df3[col] = (df3[col] - epoch).dt.days

In [ ]:
bool_columns = [
    'IsBorrowerHomeowner',
    'CurrentlyInGroup',
    'IncomeVerifiable'
]
for col in bool_columns:
    if col in df3.columns:
        df3[col] = df3[col].astype(str).str.lower()
        df3[col] = df3[col].map({'true': 1, 'false': 0})

In [ ]:
df3.info()

In [ ]:
df3.head()

In [ ]:
df3.to_csv('filled_missings_df.csv', index=True)

In [ ]:
df3

In [ ]:
continuous_variables  = [
    'ListingNumber', 'BorrowerAPR', 'BorrowerRate', 'LenderYield', 'EstimatedEffectiveYield',
    'EstimatedLoss', 'EstimatedReturn', 'EmploymentStatusDuration', 'OpenRevolvingMonthlyPayment', 'AmountDelinquent',
    'RevolvingCreditBalance', 'BankcardUtilization', 'AvailableBankcardCredit', 'TradesNeverDelinquent (percentage)',
    'ProsperPrincipalBorrowed', 'DebtToIncomeRatio', 'StatedMonthlyIncome', 'ProsperPrincipalOutstanding',
    'MonthlyLoanPayment', 'ScorexChangeAtTimeOfListing', 'LoanOriginalAmount', 'LP_CustomerPayments',
    'LP_CustomerPrincipalPayments', 'LP_InterestandFees', 'LP_ServiceFees', 'LP_CollectionFees', 'LP_GrossPrincipalLoss',
    'LP_NetPrincipalLoss', 'LP_NonPrincipalRecoverypayments', 'InvestmentFromFriendsAmount', 'PercentFunded'
]


discrete_variables  = [
    'Term', 'CreditScoreRangeLower',
    'CreditScoreRangeUpper', 'CurrentCreditLines', 'OpenCreditLines', 'TotalCreditLinespast7years',
    'OpenRevolvingAccounts', 'InquiriesLast6Months', 'TotalInquiries', 'CurrentDelinquencies',
    'DelinquenciesLast7Years', 'PublicRecordsLast10Years', 'PublicRecordsLast12Months', 'ProsperScore',
    'TotalTrades', 'TradesOpenedLast6Months', 'TotalProsperLoans', 'TotalProsperPaymentsBilled',
    'OnTimeProsperPayments', 'ProsperPaymentsLessThanOneMonthLate', 'ProsperPaymentsOneMonthPlusLate',
    'LoanCurrentDaysDelinquent', 'LoanFirstDefaultedCycleNumber', 'LoanMonthsSinceOrigination',
    'Recommendations', 'InvestmentFromFriendsCount', 'Investors', 'ListingCategory (numeric)',
    'ProsperRating (numeric)', 'IsBorrowerHomeowner', 'CurrentlyInGroup', 'DateCreditPulled',
    'IncomeVerifiable', 'LoanOriginationDate', 'FirstRecordedCreditLine', 'ClosedDate', 'ListingCreationDate'
]


categorical_variables = [
    'ListingKey', 'CreditGrade', 'LoanStatus',
    'ProsperRating (Alpha)', 'BorrowerState', 'Occupation', 'EmploymentStatus',
    'GroupKey', 'IncomeRange', 'LoanKey',
    'LoanOriginationQuarter', 'MemberKey', 'LoanNumber'
]

print(len(continuous_variables))
print(len(discrete_variables))
print(len(categorical_variables))

# TODO:
0. Wywal bezsensowe zmienne
1. Zbadać korelacje
2. Poszeregować na podstawie entropii
3. Ustalić punkt odcięcia
4. Feature selection drzewami
5. Zbuduj modele
6. Interpretacja wyników